In [1]:
import json
from urllib.request import urlopen
import pandas as pd
import sys
import passim

In [2]:
ssb_fulltext = 'O say can you see by the dawn s early light What so proudly we hail d at the twilight s last gleaming whose broad stripes and bright stars through the perilous fight O er the ramparts we watch d were so gallantly streaming And the Rockets red glare the Bombs bursting in air Gave proof through the night that our flag was still there O say does that star spangled Banner yet wave O er the Land of the free and the home of the brave On the shore dimly seen through the mists of the deep Where the foe s haughty host in dread silence reposes What is that which the breeze o er the towering steep As it fitfully blows half conceals half discloses Now it catches the gleam of the morning s first beam In full glory reflected now shines on the stream Tis the star spangled banner O long may it wave O er the land of the free and the home of the brave And where is that band who so vauntingly swore That the havoc of War and the battle s confusion A home and a country should leave us no more Their blood has wash d out their foul foot steps pollution No refuge could save the hireling and slave From the terror of flight or the gloom of the grave And the star spangled banner in triumph doth wave O er the land of the free and the home of the brave O thus be it ever when freemen shall stand Between their lov d home and the war s desolation Blest with vict ry and peace may the Heav n rescued land Praise the power that hath made and preserv d us a nation Then conquer we must when our cause it is just And this be our motto In GOD is our Trust And the star spangled banner in triumph shall wave O er the land of the free and the home of the brave'

In [2]:
bedford_inquirer_df = pd.read_csv('bedford_inquirer1861-1865.csv')

In [3]:
bedford_inquirer_df

,url,text,date,newspaper,location,no_html,ssb_present
0,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-01-04'],Bedford Inquirer,"Bedford, PA","['bedford inquirer.bedford, pa.friday burning....",no
1,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-01-04'],Bedford Inquirer,"Bedford, PA","[', tnn! inqtriren.i&quot; published ivery fri...",no
2,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-01-04'],Bedford Inquirer,"Bedford, PA","['by david over.from the london times, decembe...",no
3,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-01-04'],Bedford Inquirer,"Bedford, PA",['coughs and colds! coughs and colds ! iseason...,no
4,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-01-11'],Bedford Inquirer,"Bedford, PA",['the iktqttiiieb. fis published every friday ...,no
...,...,...,...,...,...,...,...
576,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1865-12-15'],Bedford Inquirer,"Bedford, PA","['great britain; and british ships, manned byb...",no
577,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1865-12-22'],Bedford Inquirer,"Bedford, PA",['smkxtt axd fixe woolthe lute extraordinary r...,no
578,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1865-12-22'],Bedford Inquirer,"Bedford, PA","['ihf gnqmw.bedford. pa„ fk!oa\\ dkc. 22, imki...",no
579,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1865-12-22'],Bedford Inquirer,"Bedford, PA",['iskvery friday morning.byj. r. dir borrow am...,no


In [4]:
bedford_inquirer_df = bedford_inquirer_df[bedford_inquirer_df['ssb_present'] == 'yes']
bedford_inquirer_df

,url,text,date,newspaper,location,no_html,ssb_present
59,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-04-26'],Bedford Inquirer,"Bedford, PA","['by david over.register&#39;s notice., ihtall...",yes
87,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-06-21'],Bedford Inquirer,"Bedford, PA",['vv lp*£&gt;&gt;ithe bt4r-spasigleii lltwidtb...,yes
89,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-06-28'],Bedford Inquirer,"Bedford, PA",['the star-spasgled bi^ek.ar r. s. key.oh 1 sa...,yes
94,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-07-12'],Bedford Inquirer,"Bedford, PA",['gk&quot;lls^kthe btar-sp4xgled ill^eßby p. 8...,yes
98,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-07-19'],Bedford Inquirer,"Bedford, PA",['the star-sp ixgleit h4.wer.by r. s kkt.&#39;...,yes
100,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-08-02'],Bedford Inquirer,"Bedford, PA",['tidb star-spamcided hammer.by f. s. key.oh !...,yes
104,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-08-09'],Bedford Inquirer,"Bedford, PA",['&#39; &#39; vwu v.-p &lt;§otssjc sttu-sp.i\\...,yes
110,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1861-08-16'],Bedford Inquirer,"Bedford, PA",['the star-spangled banner.by p. 8. kky.oh ! s...,yes
479,https://chroniclingamerica.loc.gov/lccn/sn8303...,<!doctype html>\n<html>\n <head>\n <...,['1865-06-30'],Bedford Inquirer,"Bedford, PA",['|ht sedfel s(t|itiferis publishedpi very fri...,yes


In [5]:
bedford_inquirer_df['CA_index'] = bedford_inquirer_df['url'].str.replace('https://chroniclingamerica.loc.gov', '', regex=False)
bedford_inquirer_df['CA_index'] = bedford_inquirer_df['CA_index'].str.replace('/ocr/', '', regex=False)

/var/folders/4g/w9bvjkdn2z73g58ttk3wndz80000gn/T/ipykernel_78406/3680222763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bedford_inquirer_df['CA_index'] = bedford_inquirer_df['url'].str.replace('https://chroniclingamerica.loc.gov', '', regex=False)
/var/folders/4g/w9bvjkdn2z73g58ttk3wndz80000gn/T/ipykernel_78406/3680222763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bedford_inquirer_df['CA_index'] = bedford_inquirer_df['CA_index'].str.replace('/ocr/', '', regex=False)


In [6]:
bedford_inquirer_df['CA_index']

59     /lccn/sn83032006/1861-04-26/ed-1/seq-1
87     /lccn/sn83032006/1861-06-21/ed-1/seq-3
89     /lccn/sn83032006/1861-06-28/ed-1/seq-4
94     /lccn/sn83032006/1861-07-12/ed-1/seq-4
98     /lccn/sn83032006/1861-07-19/ed-1/seq-4
100    /lccn/sn83032006/1861-08-02/ed-1/seq-4
104    /lccn/sn83032006/1861-08-09/ed-1/seq-4
110    /lccn/sn83032006/1861-08-16/ed-1/seq-4
479    /lccn/sn83032006/1865-06-30/ed-1/seq-1
Name: CA_index, dtype: object

In [8]:
# to direct passim toward SSB reprints, add transcription to docs
# docs = [{'id': 'star_spangled_banner', 'group': 'transcribed', 'text': ssb_fulltext}]

In [7]:
docs = []

In [8]:
for row, row in bedford_inquirer_df.iterrows():
    url = row['url']
    id = row['CA_index']

    try:
        text = urlopen(url).read().decode('utf-8')
        docs.append({'id': id, 'group': 'sn83032006', 'text': text})
    except Exception as e:
        print(f"Error fetching or decoding text from {url}: {e}")

In [9]:
with open('in.json', 'w') as f:
  for d in docs:
    print(json.dumps(d), file=f)

In [10]:
!rm -r out_cluster
!seriatim in.json out_cluster >& out_cluster.err

In [11]:
import glob, itertools

# Read one JSON record per line
def read_jsonl_file(f):
  res = []
  for line in f:
    res.append(json.loads(line))
  return res

def read_jsonl(d):
  return list(itertools.chain.from_iterable([read_jsonl_file(open(f)) for f in glob.glob(d + '/*.json')]))

In [12]:
!ls out_cluster

out.json tmp


In [13]:
read_jsonl('out_cluster/out.json')

[{'uid': -8508681740826094908,
  'cluster': 0,
  'begin': 0,
  'end': 1664,
  'boiler': False,
  'src': [{'uid': -9124707293702619645, 'begin': 1637, 'end': 3715}],
  'size': 8,
  'pboiler': 0.0,
  'group': 'sn83032006',
  'id': '/lccn/sn83032006/1861-04-26/ed-1/seq-1',
  'text': '<!DOCTYPE html>\n<html>\n    <head>\n        <title>Bedford inquirer. [volume] (Bedford, Pa.) 1857-1884, April 26, 1861, Image 1 - Chronicling America - The Library of Congress</title>\n        <link rel="stylesheet" href="/media/css/ca_print.b52ce43b3e25.css" />\n        <style type="text/css">\n            .thumbnail {\n                float: none;\n                padding-right: 1em;\n            }\n            img {\n                height: auto;\n            }\n        </style>\n        <link rel="canonical" href="/lccn/sn83032006/1861-04-26/ed-1/seq-1/" />\n        <link rel="alternate" type="application/xml" href="/lccn/sn83032006/1861-04-26/ed-1/seq-1/ocr.xml" />\n    </head>\n\n    <body id="chronam_

Identifies 12 (debatable) SSB reprintings. Some are just fragments.